In [1]:
import pandas as pd
from datetime import datetime
import calendar
import torch
import torch.nn as nn

In [2]:
data = pd.read_csv(r'D:\Code\DS108\preprocessing\tidy_data.csv', encoding = 'utf-8-sig')

In [3]:
# Số dòng và cột của dữ liệu
data.shape

(13224, 30)

In [4]:
# Các thuộc tính của dữ liệu
data.columns

Index(['name', 'brand', 'condition', 'price_old', 'price_new', 'warranty',
       'CPU', 'RAM', 'capacity', 'time', 'battery', 'screen_size',
       'operating_system', 'display_technology', 'screen_resolution', 'SIM',
       'size', 'weight', 'bluetooth', 'refresh_rate', 'GPU', 'color', 'image',
       'resolution_width', 'resolution_height', 'has_nano_sim', 'has_esim',
       'height', 'width', 'depth'],
      dtype='object')

# Tính khoảng thời gian điện thoại đã ra mắt

In [5]:
data['time'] = pd.to_datetime(data['time'])

In [6]:
# Tính toán khoảng thời gian hiện tại - 'time'
data['time_difference'] = datetime.now() - data['time']
data['time_difference'] = data['time_difference'].dt.days

In [7]:
data.drop(columns=['time'], inplace=True)

# Tính thời gian hết bảo hành

In [8]:
processed_results = []
current_date = datetime.now() # Lấy ngày và giờ hiện tại

for index, item in data['warranty'].items():
    item = str(item).strip() # Đảm bảo item là chuỗi và loại bỏ khoảng trắng

    # Kiểm tra nếu có chứa "tháng"
    if "tháng" in item:
        num_months_str = item.replace("tháng", "").strip()
        num_months = int(num_months_str)

        # Xử lý cộng tháng
        year = current_date.year
        month = current_date.month + num_months
        day = current_date.day

        while month > 12:
            month -= 12
            year += 1
                
        # Xử lý trường hợp ngày không hợp lệ sau khi cộng tháng
        try:
            new_date = datetime(year, month, day)
        except ValueError:
            # Nếu ngày không hợp lệ, lấy ngày cuối cùng của tháng đó
            last_day_of_month = calendar.monthrange(year, month)[1]
            new_date = datetime(year, month, last_day_of_month)

        # Cập nhật giá trị
        data.loc[index, 'warranty'] = new_date.strftime("%d/%m/%Y")
    else:
            datetime.strptime(item, "%d/%m/%Y")
            data.loc[index, 'warranty'] = item 

# Chuyển dữ liệu sang kiểu datetime
data['warranty'] = pd.to_datetime(data['warranty'], format='%d/%m/%Y', errors='coerce')

In [9]:
# Tạo một Series mới cho current_date
current_date_series = pd.Series([current_date] * len(data))

# Tính hiệu số ngày
time_remaining = data['warranty'] - current_date_series

# Lưu vào một cột mới
data['day_remaining_warranty'] = time_remaining.dt.days + 1

In [10]:
data.drop(columns=['warranty'], inplace=True)

# Chia dữ liệu tập train, dev và test 

In [11]:
from sklearn.model_selection import train_test_split

# Chia tập dữ liệu thành train (60%), tạm thời còn lại 40% sẽ chia tiếp
train, temp = train_test_split(data, test_size=0.4, random_state=42)

# Chia tiếp tập còn lại thành dev (20%) và test (20%)
dev, test = train_test_split(temp, test_size=0.5, random_state=42)


# Encoding

In [12]:
print(data['operating_system'].unique())

['iOS 16' 'iOS 15' 'Android 13' 'Android 14' 'iOS 17' 'ColorOS 13.1' nan
 'iOS 18' 'iOS 13' 'iOS 11' 'Android 12' 'iOS 12' 'iOS 14.1'
 'Xiaomi HyperOS' 'Funtouch OS 13' 'Android 15' 'MIUI 14' 'ColorOS 15'
 'Android 17' 'Xiaomi HyperOS 2' 'XOS 14' 'Android 11' 'ColorOS 14'
 'Android 10' 'ColorOS 12.1' 'Android 16' 'MIUI 13' 'Android T' 'OxygenOS'
 'Android 10.0' 'realme UI 4.0' 'Android U' 'Mocor' 'Android 9.0'
 'Series 30' 'S30']


In [13]:
# Hàm nhóm hệ điều hành vào các loại chính
def map_os_family(os):
    if pd.isnull(os):
        return 'Unknown'
    os = os.lower()
    if 'ios' in os:
        return 'iOS'
    elif 'android' in os:
        return 'Android'
    elif 'coloros' in os:
        return 'Android'
    elif 'miui' in os:
        return 'Android'
    elif 'hyperos' in os:
        return 'Android'
    elif 'funtouch' in os:
        return 'Android'
    elif 'oxygenos' in os:
        return 'Android'
    elif 'realme ui' in os:
        return 'Android'
    elif 'xos' in os:
        return 'Android'
    elif 'mocor' in os:
        return 'Android'
    elif 'series 30' in os or 's30' in os:
        return 'Feature OS'
    else:
        return 'Other'

# Tạo cột os
data['os'] = data['operating_system'].apply(map_os_family)




In [16]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
data['os'] = encoder.fit_transform(data['os'])

In [19]:
data['chip'] = data['CPU'].str.extract(r'(Apple|Snapdragon|MediaTek|Exynos|Unisoc)', expand=False)


In [21]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
data['chip'] = encoder.fit_transform(data['chip'])

In [25]:
data['GPU_brand'] = data['GPU'].str.extract(r'(Apple|Adreno|Mali|PowerVR|IMG|AMD|Xclipse|Immortalis)', expand=False)

from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
data['chip'] = encoder.fit_transform(data['chip'])